In [ ]:
ticker = yf.Ticker('APPL')

In [19]:
import yfinance as yf
import json

# 获取市场新闻（通过任意大型公司的代码）
ticker = yf.Ticker("^GSPC")  # 示例：标普500指数
news = ticker.news
# news 已是一个包含标题、链接、发布时间等信息的列表
# 可进一步访问链接获取全文（仍需处理详情页）

Timeout: Failed to perform, curl: (28) Connection timed out after 30012 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import json
import time
import random

class YahooFinanceNewsScraper:
    def __init__(self):
        # 初始化Chrome浏览器驱动，请确保已下载对应版本的chromedriver
        options = webdriver.ChromeOptions()
        # 可选：添加以下参数以在后台运行（无头模式）
        # options.add_argument('--headless')
        # 添加用户代理等参数模拟真实浏览器
        options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36')
        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 10)
        self.news_data = []  # 用于存储最终结果的列表

    def fetch_news_list(self, url='https://finance.yahoo.com/'):
        """获取新闻列表页，并提取新闻链接和摘要信息"""
        print("正在访问Yahoo Finance首页...")
        self.driver.get(url)
        time.sleep(random.uniform(3, 5))  # 等待页面加载，可调整时间或使用显式等待

        # 解析页面
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')

        # **关键：此处需要根据Yahoo Finance的实际HTML结构调整选择器**
        # 示例选择器（很可能需要修改）：
        # 通常新闻条目可能在特定的容器内，如具有特定类的< li >, < div >或< article >标签
        news_items = soup.select('h3 a')  # 这只是一个非常通用的示例，需要替换

        news_list = []
        for item in news_items:
            link = item.get('href')
            title = item.get_text(strip=True)
            if link and title:
                # 补全相对链接
                if link.startswith('/'):
                    link = f'https://finance.yahoo.com{link}'
                news_list.append({'title': title, 'link': link})
                print(f"找到新闻: {title}")

        print(f"共找到 {len(news_list)} 条新闻链接。")
        return news_list[:10]  # 限制爬取数量以避免请求过多

    def fetch_news_detail(self, news_item):
        """访问单条新闻详情页，提取全文和发布时间"""
        print(f"正在爬取详情: {news_item['title'][:50]}...")
        try:
            self.driver.get(news_item['link'])
            time.sleep(random.uniform(2, 4))  # 等待详情页加载

            detail_soup = BeautifulSoup(self.driver.page_source, 'html.parser')

            # **关键：提取新闻正文和发布时间**
            # 1. 提取正文：通常位于<article>、<div class="article-body">等标签内
            # 需要手动审查详情页HTML结构来确定正确的选择器
            content_div = detail_soup.find('div', class_='caas-body')  # 示例选择器，需要验证
            full_text = content_div.get_text(separator='\n', strip=True) if content_div else "正文未找到"

            # 2. 提取发布时间：可能在<time>、<div class="date">等标签内
            time_tag = detail_soup.find('time')  # 示例选择器，需要验证
            publish_time = time_tag.get('datetime') if time_tag else "时间未找到"

            news_item['full_text'] = full_text
            news_item['publish_time'] = publish_time
            news_item['crawled_time'] = time.strftime('%Y-%m-%d %H:%M:%S')

        except Exception as e:
            print(f"爬取详情页时出错: {e}")
            news_item['full_text'] = "爬取失败"
            news_item['publish_time'] = "爬取失败"

        return news_item

    def scrape_to_json(self, output_filename='yahoo_finance_news.json'):
        """主函数：执行爬取并保存为JSON"""
        news_list = self.fetch_news_list()

        for news in news_list:
            detailed_news = self.fetch_news_detail(news)
            self.news_data.append(detailed_news)
            # 随机延迟，降低请求频率
            time.sleep(random.uniform(1, 3))

        # 保存为JSON文件
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(self.news_data, f, ensure_ascii=False, indent=2)

        print(f"爬取完成！数据已保存至 {output_filename}")
        return self.news_data

    def close(self):
        """关闭浏览器"""
        self.driver.quit()

# 使用示例
if __name__ == "__main__":
    scraper = YahooFinanceNewsScraper()
    try:
        data = scraper.scrape_to_json()
        # 打印前两条数据作为预览
        for i, news in enumerate(data[:2]):
            print(f"\n--- 新闻 {i+1} 预览 ---")
            print(f"标题: {news['title']}")
            print(f"时间: {news['publish_time']}")
            print(f"正文前100字符: {news['full_text'][:100]}...")
    finally:
        scraper.close()

ModuleNotFoundError: No module named 'selenium'

In [32]:
def scrape_yahoo_news(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        resp.raise_for_status()
    except requests.RequestException as e:
        print(f"请求失败: {e}")
        return []

    soup = BeautifulSoup(resp.text, 'html.parser')
    news_items = []
    
    # 关键：这里的选择器需要根据实际网页结构调整
    # 示例：假设新闻标题在<h3>标签内，链接在<a>标签的'href'属性中
    for item in soup.select('h3 a'):  # 这只是一个示例，很可能不正确
        title = item.get_text(strip=True)
        link = item.get('href')
        if title and link:
            # 补全可能为相对链接的URL
            if link.startswith('/'):
                link = f'https://finance.yahoo.com{link}'
            news_items.append({'title': title, 'link': link})
    
    return news_items

In [ ]:
news_list = scrape_yahoo_news('https://finance.yahoo.com/topic/stock-market-news/')
for news in news_list[:5]:
    print(news)